In [ ]:

import pandas as pd
import numpy as np

from axisfuzzy.analysis.app.model import Model
from axisfuzzy.analysis.component.basic import (
    ToolNormalization,
    ToolFuzzification,
    ToolSimpleAggregation
)
from axisfuzzy.analysis.build_in import ContractCrispTable, ContractFuzzyTable
from axisfuzzy.fuzzifier import Fuzzifier

In [ ]:
my_fuzzifier = Fuzzifier(
    mf='gaussmf',
    mtype='qrofn',  # 假设目标是 q-rung orthopair fuzzy number
    pi = 0.2,
    mf_params=[{'sigma': 0.15, 'c': 0.5}]
)

In [ ]:
# --- 子模型：数据预处理流程 ---
class DataPreprocessingModel(Model):
    def get_config(self) -> dict:
        pass

    def __init__(self, norm_method: str = 'min_max'):
        super().__init__() # __init__ 现在只做初始化
        self.normalizer = ToolNormalization(method=norm_method, axis=0)
        self.fuzzifier = ToolFuzzification(fuzzifier=my_fuzzifier)

    def forward(self, data: ContractCrispTable) -> ContractFuzzyTable:
        norm_data = self.normalizer(data)
        # fuzzy_data = self.fuzzifier(norm_data)
        return norm_data

In [ ]:
class MainAnalysisModel(Model):
    def get_config(self) -> dict:
        pass

    def __init__(self):
        super().__init__() # __init__ 现在只做初始化
        self.preprocessing_pipeline = DataPreprocessingModel(norm_method='z_score')
        self.aggregator = ToolSimpleAggregation(operation='mean', axis=1)

    def forward(self, raw_input_data: ContractCrispTable):
        processed_data = self.preprocessing_pipeline(raw_input_data)
        final_scores = self.aggregator(processed_data)
        return final_scores

In [ ]:
df = pd.DataFrame(np.random.rand(5, 4), columns=[f'Feature_{i}' for i in range(4)])

In [ ]:
main_model = MainAnalysisModel()
main_model.build()

In [ ]:
main_model.summary()

In [ ]:
main_model.visualize()

In [ ]:
main_model(df)